In [1]:
!nvidia-smi

Fri Dec  6 13:46:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# clone YOLOv5 repository
%cd /content
!git clone https://github.com/ultralytics/yolov5

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 27 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (11714/11714), done.


In [3]:
%cd /content/yolov5

/content/yolov5


In [6]:
# install dependencies as necessary
!pip install -r requirements.txt
!pip uninstall wandb -qy  # deprecated dependency
import torch

from IPython.display import Image, clear_output  # to display images

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 23.8 MB/s eta 0:00:00
Setup complete. Using torch 2.5.1+cu121 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40, uuid=55950b6c-1187-479f-ef90-fe336aff9fe1, L2_cache_size=4MB)


In [4]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="3Tgt5DABjP8lKrJXfOMm")
project = rf.workspace("rajasree-vky4f").project("drowsy-uljh3")
version = project.version(1)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


In [5]:
%cat Drowsy-1/data.yaml

names:
- '0'
- '1'
nc: 2
roboflow:
  license: CC BY 4.0
  project: drowsy-uljh3
  url: https://universe.roboflow.com/rajasree-vky4f/drowsy-uljh3/dataset/1
  version: 1
  workspace: rajasree-vky4f
test: ../test/images
train: Drowsy-1/train/images
val: Drowsy-1/valid/images


In [14]:
# prompt: display number of train test and valid. not showing number just directory

import os

def count_images(directory):
  """Counts the number of image files in a directory."""
  count = 0
  for filename in os.listdir(directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
      count += 1
  return count

# Assuming your dataset is structured as follows:
train_dir = 'Drowsy-1/train/images'
valid_dir = 'Drowsy-1/valid/images'
test_dir = 'Drowsy-1/test/images'

try:
  num_train = count_images(train_dir)
  num_valid = count_images(valid_dir)
  num_test = count_images(test_dir)

  print(f"Number of training images: {num_train}")
  print(f"Number of validation images: {num_valid}")
  print(f"Number of test images: {num_test}")
except FileNotFoundError:
  print("Error: One or more image directories not found. Check the paths.")
except Exception as e:
    print(f"An error occurred: {e}")

Number of training images: 76
Number of validation images: 21
Number of test images: 11


In [22]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="API_KEY")
# project = rf.workspace().project("MODEL_ENDPOINT")
# model = project.version(VERSION).model



from roboflow import Roboflow
rf = Roboflow(api_key="3Tgt5DABjP8lKrJXfOMm")
project = rf.workspace("rajasree-vky4f").project("drowsy-uljh3")
model = project.version(1).model

# infer on a local image
print(model.predict("/content/drowsy.jpg", confidence=40, overlap=30).json())


loading Roboflow workspace...
loading Roboflow project...
{'predictions': [{'x': 546, 'y': 386, 'width': 725, 'height': 510, 'confidence': 0.7988555431365967, 'class': '1', 'class_id': 1, 'detection_id': 'f69d52b6-596f-454d-903e-ee0862b48db4', 'image_path': '/content/drowsy.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 1242, 'y': 735, 'width': 921, 'height': 665, 'confidence': 0.6810390949249268, 'class': '1', 'class_id': 1, 'detection_id': '05f63de7-7fab-4369-bbe5-afd35b2d5350', 'image_path': '/content/drowsy.jpg', 'prediction_type': 'ObjectDetectionModel'}], 'image': {'width': '2000', 'height': '1075'}}
